# Evaluation

In [1]:
import logging
import matplotlib
import multiprocessing as mp
import numpy as np
import os
import cv2
import sys
import torch
import pandas as pd
import json
import shutil
from random import sample
# Fix problem: no $DISPLAY environment variable
matplotlib.use('Agg')

from argparse import ArgumentParser
from datetime import datetime as dt
from pprint import pprint

from config import cfg
from core.test import test_net
from tensorboardX import SummaryWriter

def test(out_path=None, filename=None, test_dataset="ShapeNet", taxonomy='./datasets/ShapeNet_Chair.json', 
           rendering_path='/home/davide/Desktop/Dissertation/Data/ShapeNetRendering/%s/%s/rendering/%02d.png', 
           n_views=1, weights="./checkpoints/Pix2Vox-A.pth", samples_to_save=None):

    torch.cuda.empty_cache()
    
    # Check python version
    if sys.version_info < (3, 0):
        raise Exception("Please follow the installation instruction on 'https://github.com/hzxie/Pix2Vox'")

    # Setup logger
    mp.log_to_stderr()
    logger = mp.get_logger()
    logger.setLevel(logging.INFO)

    np.random.seed(cfg.CONST.RNG_SEED)

    # Output path
    if out_path is None:
        
        global_out_path = "./output"
        if not os.path.exists(global_out_path):
            os.makedirs(global_out_path)

        out_path = os.path.join(global_out_path, '%s', dt.now().isoformat())
    
    cfg.DIR.OUT_PATH = out_path

    # Dataset
    cfg.DATASET.TEST_DATASET = test_dataset

    if test_dataset=="ShapeNet":
        cfg.DATASETS.SHAPENET.TAXONOMY_FILE_PATH = taxonomy
        cfg.DATASETS.SHAPENET.RENDERING_PATH = rendering_path    
    elif test_dataset=="Pix3D":
        cfg.DATASETS.PIX3D.TAXONOMY_FILE_PATH = taxonomy
        cfg.DATASETS.PIX3D.RENDERING_PATH = rendering_path

    cfg.CONST.N_VIEWS_RENDERING = n_views

    # Weights
    if weights is not None and os.path.exists(weights):
        cfg.CONST.WEIGHTS = weights
    else:
        print('[FATAL] %s Please specify the file path of checkpoint.' % (dt.now()))
        sys.exit(2)

    # Print config
    print('Use config:')
    pprint(cfg)

    # Set GPU to use
    if type(cfg.CONST.DEVICE) == str:
        os.environ["CUDA_VISIBLE_DEVICES"] = cfg.CONST.DEVICE

    log_dir = out_path % 'logs'
    test_writer = SummaryWriter(os.path.join(log_dir, 'test'))

    # Start train process
    mean_iou, pandas_rows = test_net(cfg, output_dir=out_path, filename=filename, test_writer=test_writer, samples_to_save=samples_to_save)
    return mean_iou


def evaluate(rendering_paths, datasets, ckpts, out_path, test_dataset="ShapeNet", taxonomy='./datasets/ShapeNet_Test.json', 
             samples_to_save=None, save_input_images=None, n_views=1):
    
    pandas_rows = []
    pandas_dir = out_path % 'pandas'

    if not os.path.exists(pandas_dir):
        os.makedirs(pandas_dir)
    columns = ["Dataset", "Checkpoint", "Max IoU", "t=0.20", "t=0.30", "t=0.40", "t=0.50"]
    
    img_dir = out_path % 'images'
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)
    
    if(save_input_images):
        save_input_images(samples_to_save, img_dir, rendering_paths)

    for ckpt in ckpts:
        
        temp_pandas_rows = []

        for i in range(len(rendering_paths)):
            
            ckpt_name = ckpt.split(".")[0]
            weights = os.path.join("./checkpoints", ckpt)
            filename = f"{ckpt_name}_{datasets[i]}"+"_%s_%s_voxel.png"
            
            mean_iou = test(out_path=out_path, filename=filename, test_dataset=test_dataset, rendering_path=rendering_paths[i], 
                            weights=weights, taxonomy=taxonomy, samples_to_save=samples_to_save, n_views=n_views)            
            
            max_iou = np.max(mean_iou)
            mean_iou = mean_iou.tolist()
            temp_pandas_rows.append((datasets[i], ckpt_name, max_iou, mean_iou[0], mean_iou[1], mean_iou[2], mean_iou[3]))

        df_temp = pd.DataFrame(temp_pandas_rows, columns = columns)
        df_temp.to_csv(os.path.join(pandas_dir,f"{ckpt}_evaluation.csv"), index=False)

        pandas_rows.extend(temp_pandas_rows)

    df = pd.DataFrame(pandas_rows, columns = columns)
    df.to_csv(os.path.join(pandas_dir,"ALL_evaluation.csv"), index=False)


def sample_model_ids(taxonomy_file, n_models):
    taxonomies = []
    with open(taxonomy_file, encoding='utf-8') as file:
        taxonomies = json.loads(file.read())
    taxonomies = {t['taxonomy_id']: t for t in taxonomies}

    model_ids = taxonomies["03001627"]["test"]
    return sample(model_ids, n_models)

### ShapeNet

In [ ]:
def save_input_images(samples_to_save, img_dir, rendering_paths):
    for sample in samples_to_save:
        for i in range(len(rendering_paths)):
            # Inpaint
            filename = f"{datasets[i]}_{sample}_image.png"

            render_path = rendering_paths[i] % ("03001627", sample, 0)
            shutil.copy(render_path, os.path.join(img_dir,filename))      

datasets = ["ShapeNet"]
rendering_paths = [os.path.join(f'/home/davide/Desktop/Dissertation/Data/{dataset}Rendering','%s','%s','rendering','%02d.png') for dataset in datasets]
ckpts = os.listdir("/home/davide/Desktop/Dissertation/Models/Pix2Vox/checkpoints")
out_path = os.path.join("./output-ShapeNet", '%s')
taxonomy='./datasets/ShapeNet_Chair.json'

# Debug
# ckpts = ["Pix2Vox-A.pth"]
# taxonomy='./datasets/ShapeNet_Test.json'

# samples_to_save = sample_model_ids(taxonomy, n_models=2)
samples_to_save = ['3a0e392db610f1a1504d5af97121b5f','131edf0948b60ee6372c8cd7d07d8ddc', '942a10a33418205819fb4103277a6b93','d97c5945e9449a58737e4e0df09d751']

evaluate(rendering_paths=rendering_paths, datasets=datasets, ckpts=ckpts, out_path=out_path, taxonomy=taxonomy, samples_to_save=samples_to_save, 
         save_input_images=save_input_images)

### Evaluate ORE_Random w/o inpainting

In [ ]:
def save_input_images(samples_to_save, img_dir, rendering_paths, gt_path='/home/davide/Desktop/Dissertation/Data/ShapeNetRendering/%s/%s/rendering/%02d.png'):
    for sample in samples_to_save:
        for i in range(len(rendering_paths)):
            # GT
            gt_render_path = gt_path % ("03001627", sample, 0)
            gt_filename = f"GT_{sample}_image.png"
            shutil.copy(gt_render_path, os.path.join(img_dir,gt_filename))

            img = cv2.imread(gt_render_path, cv2.IMREAD_UNCHANGED)
            
            # MASK
            render_path = rendering_paths[i] % ("03001627", sample, 0)
            render_path = render_path.replace(".png","_mask.png")
            
            mask_filename = f"{datasets[i]}_{sample}_mask.png"
            shutil.copy(render_path, os.path.join(img_dir,mask_filename))

            mask = cv2.imread(render_path, cv2.IMREAD_UNCHANGED)
            mask = cv2.bitwise_not(mask)

            # finding the MASK contours
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            # take bb
            cnt = max(contours, key = cv2.contourArea)
            mask_left, mask_top, bb_w, bb_h = cv2.boundingRect(cnt)
            mask_right = mask_left+bb_w
            mask_bot = mask_top+bb_h

            # mask rectangle and border
            img_cnt = img.copy()
            alpha = 0.4
            
            cv2.rectangle(img_cnt, (mask_left, mask_top), (mask_right, mask_bot), (0,0,219,255), -2)
            img_cnt = cv2.addWeighted(img_cnt, alpha, img, 1 - alpha, 0)
            cv2.rectangle(img_cnt, (mask_left, mask_top), (mask_right, mask_bot), (0,0,219,255), 1)

            filename = f"{datasets[i]}_{sample}_image.png"
            cv2.imwrite(os.path.join(img_dir,filename), img_cnt)

datasets = ["ORE_Random_15", "ORE_Random_25", "ORE_Random_50"]
rendering_paths = [os.path.join('/home/davide/Desktop/Dissertation/Data/OccludedShapeNet', dataset,'%s','%s','rendering','%02d.png') for dataset in datasets]
ckpts = os.listdir("/home/davide/Desktop/Dissertation/Models/Pix2Vox/checkpoints")
out_path = os.path.join("./output-zzz", '%s')
taxonomy='./datasets/ShapeNet_Chair.json'

# ckpts = ["Pix2Vox-A.pth"]
# taxonomy='./datasets/ShapeNet_Test.json'

samples_to_save = sample_model_ids(taxonomy, n_models=2)

evaluate(rendering_paths=rendering_paths, datasets=datasets, ckpts=ckpts, out_path=out_path, taxonomy=taxonomy, samples_to_save=samples_to_save, 
         save_input_images=save_input_images, n_views=2)

### Evaluate ORE_Random single-image inpainting

In [ ]:
def save_input_images(samples_to_save, img_dir, rendering_paths):
    for sample in samples_to_save:
        for i in range(len(rendering_paths)):
            # Inpaint
            filename = f"{datasets[i]}_{sample}_inpainted.png"

            render_path = rendering_paths[i] % ("03001627", sample, 0)
            shutil.copy(render_path, os.path.join(img_dir,filename))      

datasets = ["ORE_Random_Inpainting_15", "ORE_Random_Inpainting_25", "ORE_Random_Inpainting_50"]
rendering_paths = [os.path.join('/home/davide/Desktop/Dissertation/Data/Inpainting', dataset,'%s','%s','rendering','%02d.png') for dataset in datasets]
ckpts = os.listdir("/home/davide/Desktop/Dissertation/Models/Pix2Vox/checkpoints")
out_path = os.path.join("./output-ORE_Random_SingleView_Inpainting", '%s')
taxonomy='./datasets/ShapeNet_Chair.json'

# Debug
# ckpts = ["Pix2Vox-A.pth"]
# taxonomy='./datasets/ShapeNet_Test.json'

# samples_to_save = sample_model_ids(taxonomy, n_models=2)
samples_to_save = ['3a0e392db610f1a1504d5af97121b5f','131edf0948b60ee6372c8cd7d07d8ddc', '942a10a33418205819fb4103277a6b93','d97c5945e9449a58737e4e0df09d751']

evaluate(rendering_paths=rendering_paths, datasets=datasets, ckpts=ckpts, out_path=out_path, taxonomy=taxonomy, samples_to_save=samples_to_save, 
         save_input_images=save_input_images)

#### Evaluate ORE_Random multi-image inpainting

In [5]:
def save_input_images(samples_to_save, img_dir, rendering_paths):
    for sample in samples_to_save:
        for i in range(len(rendering_paths)):           
            # Inpaint
            filename = f"{datasets[i]}_{sample}_inpainted.png"

            render_path = rendering_paths[i] % ("03001627", sample, 0)
            shutil.copy(render_path, os.path.join(img_dir,filename))
        

datasets = ["ORE_Random_Inpainting_15", "ORE_Random_Inpainting_25", "ORE_Random_Inpainting_50"]
datasets = ["ORE_Random_Inpainting_15"]
rendering_paths = [os.path.join('/home/davide/Desktop/Dissertation/Data/TEST_Multi', dataset,'%s','%s','rendering','%02d.png') for dataset in datasets]
ckpts = os.listdir("/home/davide/Desktop/Dissertation/Models/Pix2Vox/checkpoints")
out_path = os.path.join("./output-TEST", '%s')
taxonomy='./datasets/ShapeNet_Chair.json'

# Debug
# ckpts = ["Pix2Vox-A.pth"]
# taxonomy='./datasets/ShapeNet_Test.json'

# samples_to_save = sample_model_ids(taxonomy, n_models=2)
samples_to_save = ['3a0e392db610f1a1504d5af97121b5f','131edf0948b60ee6372c8cd7d07d8ddc', '942a10a33418205819fb4103277a6b93','d97c5945e9449a58737e4e0df09d751']

evaluate(rendering_paths=rendering_paths, datasets=datasets, ckpts=ckpts, out_path=out_path, taxonomy=taxonomy, samples_to_save=samples_to_save, 
         save_input_images=save_input_images, n_views=1)

Use config:
{'CONST': {'BATCH_SIZE': 32,
           'CROP_IMG_H': 128,
           'CROP_IMG_W': 128,
           'DEVICE': '0',
           'IMG_H': 224,
           'IMG_W': 224,
           'MAX_ERASING': 20,
           'MIN_ERASING': 10,
           'N_VIEWS_RENDERING': 1,
           'N_VOX': 32,
           'P_ERASING': 50,
           'RNG_SEED': 0,
           'WEIGHTS': './checkpoints/Pix2Vox-A.pth'},
 'DATASET': {'MEAN': [0.5, 0.5, 0.5],
             'STD': [0.5, 0.5, 0.5],
             'TEST_DATASET': 'ShapeNet',
             'TRAIN_DATASET': 'ShapeNet'},
 'DATASETS': {'PASCAL3D': {'ANNOTATION_PATH': '/home/hzxie/Datasets/PASCAL3D/Annotations/%s_imagenet/%s.mat',
                           'RENDERING_PATH': '/home/hzxie/Datasets/PASCAL3D/Images/%s_imagenet/%s.JPEG',
                           'TAXONOMY_FILE_PATH': './datasets/Pascal3D.json',
                           'VOXEL_PATH': '/home/hzxie/Datasets/PASCAL3D/CAD/%s/%02d.binvox'},
              'PIX3D': {'ANNOTATION_PATH': '/home/d

/home/davide/miniconda3/envs/pix2vox/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/davide/miniconda3/envs/pix2vox/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


./checkpoints/Pix2Vox-A.pth
[INFO] 2023-08-12 19:07:57.008491 Loading weights from ./checkpoints/Pix2Vox-A.pth ...


[INFO/Process-4] child process calling self.run()
[INFO/Process-4] child process calling self.run()
[INFO/Process-4] child process calling self.run()
[INFO/Process-4] child process calling self.run()
[INFO/Process-4] created temp directory /tmp/pymp-_v5_nht1
[INFO/Process-4] created temp directory /tmp/pymp-_v5_nht1
[INFO/Process-4] created temp directory /tmp/pymp-_v5_nht1
[INFO/Process-4] created temp directory /tmp/pymp-_v5_nht1


[INFO] 2023-08-12 19:07:57.432699 Test[1/1357] Taxonomy = 03001627 Sample = 3b3a9f4e3aa9f2f4d39a194653571dfc EDLoss = 1.4152 RLoss = 1.2042 IoU = ['0.3680', '0.3896', '0.3818', '0.3343']
[INFO] 2023-08-12 19:07:57.453262 Test[2/1357] Taxonomy = 03001627 Sample = a691eee4545ce2fade94aad0562ac2e EDLoss = 10.4047 RLoss = 10.9965 IoU = ['0.2608', '0.2441', '0.2258', '0.2079']
[INFO] 2023-08-12 19:07:57.471680 Test[3/1357] Taxonomy = 03001627 Sample = a58f8f1bd61094b3ff2c92c2a4f65876 EDLoss = 2.5354 RLoss = 2.5789 IoU = ['0.1143', '0.1034', '0.0956', '0.0795']
[INFO] 2023-08-12 19:07:57.491284 Test[4/1357] Taxonomy = 03001627 Sample = 27c00ec2b6ec279958e80128fd34c2b1 EDLoss = 0.6791 RLoss = 0.7381 IoU = ['0.5841', '0.5743', '0.5672', '0.5423']
[INFO] 2023-08-12 19:07:57.510404 Test[5/1357] Taxonomy = 03001627 Sample = 484f0070df7d5375492d9da2668ec34c EDLoss = 0.9262 RLoss = 0.8053 IoU = ['0.4335', '0.4340', '0.4197', '0.4020']
[INFO] 2023-08-12 19:07:57.529785 Test[6/1357] Taxonomy = 030016

[INFO/Process-4] process shutting down
[INFO/Process-4] process shutting down
[INFO/Process-4] process shutting down
[INFO/Process-4] process shutting down
[INFO/Process-4] process exiting with exitcode 0
[INFO/Process-4] process exiting with exitcode 0
[INFO/Process-4] process exiting with exitcode 0
[INFO/Process-4] process exiting with exitcode 0


[INFO] 2023-08-12 19:08:22.723964 Test[1351/1357] Taxonomy = 03001627 Sample = 4f0fc7761771d073fe2cac60a76c70a EDLoss = 1.1867 RLoss = 1.4169 IoU = ['0.1066', '0.1054', '0.1075', '0.1086']
[INFO] 2023-08-12 19:08:22.741238 Test[1352/1357] Taxonomy = 03001627 Sample = 94f9fa9f8a20f7d040ef1c8b63a628f9 EDLoss = 1.4383 RLoss = 1.5472 IoU = ['0.4523', '0.4093', '0.3564', '0.2944']
[INFO] 2023-08-12 19:08:22.760726 Test[1353/1357] Taxonomy = 03001627 Sample = e3a7c517fbe992ddd493bfe20f94b6ab EDLoss = 1.3305 RLoss = 1.4902 IoU = ['0.0241', '0.0162', '0.0137', '0.0120']
[INFO] 2023-08-12 19:08:22.777816 Test[1354/1357] Taxonomy = 03001627 Sample = 9233077bbe6926c239465fa20b0ba7fb EDLoss = 1.8202 RLoss = 1.6152 IoU = ['0.5343', '0.5179', '0.4725', '0.3708']
[INFO] 2023-08-12 19:08:22.795394 Test[1355/1357] Taxonomy = 03001627 Sample = 51d67f7174881679f256a8c3e3396495 EDLoss = 0.7707 RLoss = 0.5998 IoU = ['0.8371', '0.8338', '0.8271', '0.7897']
[INFO] 2023-08-12 19:08:22.811806 Test[1356/1357] T

### Evaluation on Pix3D dataset

In [18]:
def save_input_images(samples_to_save, img_dir, rendering_paths, gt_path='/home/davide/Desktop/Dissertation/Data/ShapeNetRendering/%s/%s/rendering/%02d.png'):
    for sample in samples_to_save:
        for i in range(len(rendering_paths)):
            # GT
            gt_render_path = gt_path % ("chair", sample, "png")
            gt_filename = f"GT_{sample}_image.png"
            shutil.copy(gt_render_path, os.path.join(img_dir,gt_filename))


def sample_model_ids(taxonomy_file, n_models):
    taxonomies = []
    with open(taxonomy_file, encoding='utf-8') as file:
        taxonomies = json.loads(file.read())
    taxonomies = {t['taxonomy_id']: t for t in taxonomies}

    model_ids = taxonomies["chair"]["test"]
    return sample(model_ids, n_models)


datasets = ["Pix3D"]
rendering_paths = ['/home/davide/Desktop/Dissertation/Data/Pix3D/img/%s/%s.%s']
ckpts = os.listdir("/home/davide/Desktop/Dissertation/Models/Pix2Vox/checkpoints")
out_path = os.path.join("./output-Pix3D", '%s')
taxonomy='./datasets/Pix3D.json'

# ckpts = ["Pix2Vox-A.pth"]

samples_to_save = sample_model_ids(taxonomy, n_models=4)

evaluate(rendering_paths=rendering_paths, datasets=datasets, ckpts=ckpts, out_path=out_path, test_dataset="Pix3D",  taxonomy=taxonomy, samples_to_save=samples_to_save)

Use config:
{'CONST': {'BATCH_SIZE': 32,
           'CROP_IMG_H': 128,
           'CROP_IMG_W': 128,
           'DEVICE': '0',
           'IMG_H': 224,
           'IMG_W': 224,
           'MAX_ERASING': 20,
           'MIN_ERASING': 10,
           'N_VIEWS_RENDERING': 1,
           'N_VOX': 32,
           'P_ERASING': 50,
           'RNG_SEED': 0,
           'WEIGHTS': './checkpoints/Erasing_10-40.pth'},
 'DATASET': {'MEAN': [0.5, 0.5, 0.5],
             'STD': [0.5, 0.5, 0.5],
             'TEST_DATASET': 'Pix3D',
             'TRAIN_DATASET': 'ShapeNet'},
 'DATASETS': {'PASCAL3D': {'ANNOTATION_PATH': '/home/hzxie/Datasets/PASCAL3D/Annotations/%s_imagenet/%s.mat',
                           'RENDERING_PATH': '/home/hzxie/Datasets/PASCAL3D/Images/%s_imagenet/%s.JPEG',
                           'TAXONOMY_FILE_PATH': './datasets/Pascal3D.json',
                           'VOXEL_PATH': '/home/hzxie/Datasets/PASCAL3D/CAD/%s/%02d.binvox'},
              'PIX3D': {'ANNOTATION_PATH': '/home/

[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process calling self.run()
[INFO/Process-77] child process

[INFO] 2023-08-11 11:42:51.229733 Test[1/2553] Taxonomy = chair Sample = 0118 EDLoss = 1.7961 RLoss = 1.6403 IoU = ['0.4201', '0.3949', '0.3427', '0.2558']
[INFO] 2023-08-11 11:42:51.779115 Test[2/2553] Taxonomy = chair Sample = 0119 EDLoss = 2.1574 RLoss = 2.2352 IoU = ['0.3536', '0.3267', '0.2842', '0.2483']
[INFO] 2023-08-11 11:42:52.263013 Test[3/2553] Taxonomy = chair Sample = 0120 EDLoss = 4.4843 RLoss = 3.4362 IoU = ['0.1446', '0.1495', '0.1541', '0.1520']
[INFO] 2023-08-11 11:42:52.748011 Test[4/2553] Taxonomy = chair Sample = 0121 EDLoss = 2.8835 RLoss = 3.0045 IoU = ['0.1387', '0.1797', '0.2448', '0.2675']
[INFO] 2023-08-11 11:42:53.202089 Test[5/2553] Taxonomy = chair Sample = 0122 EDLoss = 2.4402 RLoss = 2.2674 IoU = ['0.3591', '0.3185', '0.2688', '0.2097']
[INFO] 2023-08-11 11:42:53.659251 Test[6/2553] Taxonomy = chair Sample = 0124 EDLoss = 2.4861 RLoss = 2.4935 IoU = ['0.3564', '0.3420', '0.3196', '0.2840']
[INFO] 2023-08-11 11:42:54.121864 Test[7/2553] Taxonomy = chair 

libpng warning: iCCP: known incorrect sRGB profile


[INFO] 2023-08-11 11:54:01.947797 Test[2065/2553] Taxonomy = chair Sample = 2991 EDLoss = 1.9354 RLoss = 2.1177 IoU = ['0.2735', '0.2501', '0.2192', '0.1966']
[INFO] 2023-08-11 11:54:01.964709 Test[2066/2553] Taxonomy = chair Sample = 2992 EDLoss = 2.8326 RLoss = 3.1834 IoU = ['0.2089', '0.2164', '0.2197', '0.2195']
[INFO] 2023-08-11 11:54:01.981047 Test[2067/2553] Taxonomy = chair Sample = 2993 EDLoss = 2.3774 RLoss = 2.4906 IoU = ['0.1653', '0.1915', '0.1817', '0.1261']
[INFO] 2023-08-11 11:54:02.133426 Test[2068/2553] Taxonomy = chair Sample = 2994 EDLoss = 2.1912 RLoss = 2.5079 IoU = ['0.1561', '0.1200', '0.0998', '0.0852']
[INFO] 2023-08-11 11:54:02.165334 Test[2069/2553] Taxonomy = chair Sample = 2995 EDLoss = 3.1638 RLoss = 2.5995 IoU = ['0.1625', '0.2025', '0.2438', '0.2994']
[INFO] 2023-08-11 11:54:02.181768 Test[2070/2553] Taxonomy = chair Sample = 2996 EDLoss = 2.6779 RLoss = 2.7118 IoU = ['0.2477', '0.2404', '0.2033', '0.1517']
[INFO] 2023-08-11 11:54:02.205512 Test[2071/25

...

[INFO] 2023-08-11 12:27:36.674195 Test[1/2553] Taxonomy = chair Sample = 0118 EDLoss = 1.7999 RLoss = 1.7490 IoU = ['0.4178', '0.3847', '0.3374', '0.2800']
[INFO] 2023-08-11 12:27:37.417258 Test[2/2553] Taxonomy = chair Sample = 0119 EDLoss = 2.3024 RLoss = 2.4361 IoU = ['0.3620', '0.3321', '0.2974', '0.2689']
[INFO] 2023-08-11 12:27:37.952576 Test[3/2553] Taxonomy = chair Sample = 0120 EDLoss = 5.1620 RLoss = 3.6873 IoU = ['0.1479', '0.1525', '0.1576', '0.1580']
[INFO] 2023-08-11 12:27:38.481217 Test[4/2553] Taxonomy = chair Sample = 0121 EDLoss = 3.0293 RLoss = 2.6048 IoU = ['0.1859', '0.2194', '0.2485', '0.2602']
[INFO] 2023-08-11 12:27:38.976437 Test[5/2553] Taxonomy = chair Sample = 0122 EDLoss = 2.7329 RLoss = 2.5771 IoU = ['0.3502', '0.3139', '0.2837', '0.2338']
[INFO] 2023-08-11 12:27:39.523809 Test[6/2553] Taxonomy = chair Sample = 0124 EDLoss = 2.6566 RLoss = 2.7482 IoU = ['0.3344', '0.3139', '0.2911', '0.2639']
[INFO] 2023-08-11 12:27:40.063546 Test[7/2553] Taxonomy = chair 

libpng warning: iCCP: known incorrect sRGB profile


[INFO] 2023-08-11 12:38:46.446829 Test[2063/2553] Taxonomy = chair Sample = 2984 EDLoss = 2.8059 RLoss = 2.8152 IoU = ['0.1426', '0.1315', '0.1076', '0.0905']
[INFO] 2023-08-11 12:38:46.465386 Test[2064/2553] Taxonomy = chair Sample = 2989 EDLoss = 1.9554 RLoss = 1.6354 IoU = ['0.2689', '0.2005', '0.0539', '0.0053']
[INFO] 2023-08-11 12:38:46.483212 Test[2065/2553] Taxonomy = chair Sample = 2991 EDLoss = 1.8647 RLoss = 1.6890 IoU = ['0.3299', '0.3075', '0.2670', '0.2287']
[INFO] 2023-08-11 12:38:46.501057 Test[2066/2553] Taxonomy = chair Sample = 2992 EDLoss = 2.7874 RLoss = 3.2373 IoU = ['0.2055', '0.2152', '0.2168', '0.2298']
[INFO] 2023-08-11 12:38:46.519426 Test[2067/2553] Taxonomy = chair Sample = 2993 EDLoss = 2.6512 RLoss = 3.0480 IoU = ['0.1199', '0.1175', '0.1158', '0.1013']
[INFO] 2023-08-11 12:38:46.695611 Test[2068/2553] Taxonomy = chair Sample = 2994 EDLoss = 2.1023 RLoss = 2.1684 IoU = ['0.1992', '0.1632', '0.1096', '0.0707']
[INFO] 2023-08-11 12:38:46.731598 Test[2069/25

[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down
[INFO/Process-81] process shutting down


[INFO] 2023-08-11 12:39:00.509259 Test[2550/2553] Taxonomy = chair Sample = 3833 EDLoss = 10.9865 RLoss = 12.1798 IoU = ['0.2774', '0.2679', '0.2590', '0.2516']
[INFO] 2023-08-11 12:39:00.526331 Test[2551/2553] Taxonomy = chair Sample = 3834 EDLoss = 16.8477 RLoss = 13.6290 IoU = ['0.1414', '0.1212', '0.0985', '0.0770']
[INFO] 2023-08-11 12:39:00.542900 Test[2552/2553] Taxonomy = chair Sample = 3838 EDLoss = 17.1199 RLoss = 13.7426 IoU = ['0.2883', '0.2800', '0.2694', '0.2495']
[INFO] 2023-08-11 12:39:00.559749 Test[2553/2553] Taxonomy = chair Sample = 3839 EDLoss = 16.7127 RLoss = 14.6927 IoU = ['0.2789', '0.2708', '0.2579', '0.2384']

============================ TEST RESULTS ============================
Taxonomy	#Sample	Baseline	t=0.20	t=0.30	t=0.40	t=0.50	
chair   	2553	N/a		0.2080	0.1906	0.1669	0.1382	

Overall [0.20800469401036234, 0.1906085779752571, 0.1668749044771133, 0.13815179241868275]

